In [17]:
from z3 import *
import csv

In [27]:
# basic puzzle
word_length = 5

def define_letter_variables():
    return [z3.Int(f"letter_{index}") for index in range(word_length)]

def add_alphabet_modeling_constraints(solver, letter_vars):
    for letter_var in letter_vars:
        solver.add(letter_var >= 0, letter_var <= 25)

    return solver

letter_to_index_map = {letter: index for index, letter in enumerate("abcdefghijklmnopqrstuvwxyz")}
index_to_letter_map = {index: letter for letter, index in letter_to_index_map.items()}

def pretty_print_solution(model, letter_vars):
    word = []
    for index, letter_var in enumerate(letter_vars):
        word.append(index_to_letter_map[model[letter_var].as_long()])

    print(''.join(word))

# dictionary
def load_dictionary():
    dictionary = []
    with open('words.csv', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        dictionary = list(map(lambda x: x[0].strip().lower(), spamreader))
    return dictionary

def add_legal_words_constraints(solver, words, letter_vars):
    all_words_disjunction = []

    for word in words:
        word_conjuction = z3.And([letter_vars[index] == letter_to_index_map[letter] for index, letter in enumerate(word)])
        all_words_disjunction.append(word_conjuction)

    solver.add(z3.Or(all_words_disjunction))

    return solver

# letter frequency
def add_doesnt_contain_letter_constraint(solver, letter_vars, letter):
    for letter_var in letter_vars:
        solver.add(letter_var != letter_to_index_map[letter])

    return solver

def add_contains_letter_constraint(solver, letter_vars, letter):
    solver.add(z3.Or([letter_var == letter_to_index_map[letter] for letter_var in letter_vars]))

    return solver

def add_invalid_position_constraint(solver, letter_vars, letter, position):
    solver.add(letter_vars[position] != letter_to_index_map[letter])

    return solver

def solvingModel(solver, step):
    print("[SOLVING] " + step)
    result = solver.check()
    print(result)

    model = solver.model()
    pretty_print_solution(model, letter_vars)

# subsequent guesses
def add_exact_letter_position_constraint(solver, letter_vars, letter, position):
    solver.add(letter_vars[position] == letter_to_index_map[letter])

    return solver

def add_letter_appears_once_constraint(solver, letter_vars, letter):
    unique_letter_disjunction = []

    for letter_var in letter_vars:
        this_letter_conjunction = [letter_var == letter_to_index_map[letter]]
        for other_letter_var in letter_vars:
            if letter_var == other_letter_var:
                continue
            this_letter_conjunction.append(other_letter_var != letter_to_index_map[letter])
        unique_letter_disjunction.append(z3.And(this_letter_conjunction))

    solver.add(z3.Or(unique_letter_disjunction))

    return solver

# friendlier controlls
def green(solver, letter_vars, letter, position):
    add_exact_letter_position_constraint(solver, letter_vars, letter, position)

def yellow(solver, letter_vars, letter, position):
    add_letter_appears_once_constraint(solver, letter_vars, letter)
    add_invalid_position_constraint(solver, letter_vars, letter, position)

def grey(solver, letter_vars, letter, position):
    add_doesnt_contain_letter_constraint(solver, letter_vars, letter)

def second_grey(solver, letter_vars, letter, position):
    add_invalid_position_constraint(solver, letter_vars, letter, position)


if __name__ == "__main__":
    solver = Solver()
    letter_vars = define_letter_variables()
    solver = add_alphabet_modeling_constraints(solver, letter_vars)
    # sanity checked
    words = load_dictionary() 
    solver = add_legal_words_constraints(solver, words, letter_vars)
    # for banned_letter in "badg":
        # solver = add_doesnt_contain_letter_constraint(solver, letter_vars, banned_letter)

    solver = add_contains_letter_constraint(solver, letter_vars, "e")

    solvingModel(solver, "with frequency")



[SOLVING] with frequency
sat
scope
